# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
mens = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
mens['League'] = 'M'
mens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,117748.000000,117748.000000,117748.00000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,...,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000
mean,2014.146355,70.294986,1288.25451,75.878936,1283.138830,63.888287,0.068689,26.401824,55.760242,7.347445,...,20.159790,12.073403,17.732454,10.461740,21.625650,11.409722,13.888907,6.004739,2.868185,19.305780
std,6.515929,35.772556,105.34750,10.998547,104.795432,10.848767,0.305098,4.680314,7.456374,3.119260,...,6.068136,5.344049,7.081056,4.221039,4.518197,3.724567,4.382700,2.745969,2.019050,4.553353
min,2003.000000,0.000000,1101.00000,34.000000,1101.000000,20.000000,0.000000,10.000000,26.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,2009.000000,40.000000,1199.00000,68.000000,1192.000000,57.000000,0.000000,23.000000,51.000000,5.000000,...,16.000000,8.000000,13.000000,7.000000,19.000000,9.000000,11.000000,4.000000,1.000000,16.000000
50%,2014.000000,73.000000,1287.00000,75.000000,1282.000000,64.000000,0.000000,26.000000,55.000000,7.000000,...,20.000000,12.000000,17.000000,10.000000,21.000000,11.000000,14.000000,6.000000,3.000000,19.000000
75%,2020.000000,101.000000,1381.00000,83.000000,1374.000000,71.000000,0.000000,29.000000,60.000000,9.000000,...,24.000000,15.000000,22.000000,13.000000,25.000000,14.000000,17.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,1480.00000,149.000000,1480.000000,144.000000,6.000000,57.000000,103.000000,26.000000,...,59.000000,48.000000,65.000000,36.000000,49.000000,31.000000,41.000000,22.000000,18.000000,45.000000


In [3]:
womens = pd.read_csv('data/WRegularSeasonDetailedResults.csv')
womens['League'] = 'W'
womens.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,...,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000,80626.000000
mean,2017.404609,69.183626,3285.116823,71.706633,3286.689554,57.242044,0.051708,25.847034,58.966574,6.276077,...,17.918413,10.507392,15.503808,11.360020,22.422122,10.935852,17.150745,7.109977,2.820839,18.192990
std,4.582659,36.157922,104.073477,11.536993,105.505327,10.960867,0.259072,4.978157,7.969144,3.127369,...,6.456006,4.936838,6.630184,4.640191,4.936106,3.805935,5.277180,3.192300,2.062848,4.556919
min,2010.000000,0.000000,3101.000000,30.000000,3101.000000,11.000000,0.000000,9.000000,30.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,3.000000
25%,2013.000000,36.000000,3196.000000,64.000000,3195.000000,50.000000,0.000000,22.000000,53.000000,4.000000,...,13.000000,7.000000,11.000000,8.000000,19.000000,8.000000,13.000000,5.000000,1.000000,15.000000
50%,2017.000000,73.000000,3283.000000,71.000000,3287.000000,57.000000,0.000000,25.000000,59.000000,6.000000,...,17.000000,10.000000,15.000000,11.000000,22.000000,11.000000,17.000000,7.000000,2.000000,18.000000
75%,2022.000000,101.000000,3376.000000,79.000000,3377.000000,64.000000,0.000000,29.000000,64.000000,8.000000,...,22.000000,14.000000,20.000000,14.000000,26.000000,13.000000,20.000000,9.000000,4.000000,21.000000
max,2025.000000,132.000000,3480.000000,140.000000,3480.000000,130.000000,5.000000,58.000000,113.000000,30.000000,...,80.000000,37.000000,52.000000,38.000000,53.000000,34.000000,49.000000,26.000000,21.000000,47.000000


The IDs are definitely distinct so we can combine into a single dataframe

In [4]:
data = pd.concat([mens, womens])

Get the distinct team/Years

In [5]:
teams = pd.concat([data[['WTeamID', 'Season', 'League']].rename(columns={'WTeamID': 'TeamID'}),
                   data[['LTeamID', 'Season', 'League']].rename(columns={'LTeamID': 'TeamID'})]).drop_duplicates().reset_index()

teamMapping = {(x.TeamID, x.Season): x.Index for x in teams.itertuples()}

And get the distinct programs

In [6]:
programs = teams.TeamID.drop_duplicates().reset_index()

programMapping = {x.TeamID: x.Index for x in programs.itertuples()}

Define the training data. The x's will be the indexes of two team IDs and their average stats, the y's will be the score difference.

In [7]:
winningStats = data[['WTeamID', 'Season', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
                    'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
winningStats.columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 
                      'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
losingStats = data[['LTeamID', 'Season', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 
                             'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
losingStats.columns=['TeamID', 'Season', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 
                     'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
teamStats = pd.concat([winningStats, losingStats]).groupby(['TeamID', 'Season']).mean()

In [8]:
def gen_dataset(data):
    winning_team = data.apply(lambda x: teamMapping[(x.WTeamID, x.Season)], axis=1)
    losing_team = data.apply(lambda x: teamMapping[(x.LTeamID, x.Season)], axis=1)
    winning_program = data.apply(lambda x: programMapping[x.WTeamID], axis=1)
    losing_program = data.apply(lambda x: programMapping[x.LTeamID], axis=1)
    winner_stats = data.apply(lambda x: teamStats.loc[(x.WTeamID, x.Season)], axis=1)
    loser_stats = data.apply(lambda x: teamStats.loc[(x.LTeamID, x.Season)], axis=1)
    winning_matchups = np.concatenate([np.stack([winning_program, winning_team, losing_program, losing_team], axis=1),
                                       winner_stats, loser_stats], axis=1)
    losing_matchups = np.concatenate([np.stack([losing_program, losing_team, winning_program, winning_team], axis=1),
                                      loser_stats, winner_stats], axis=1)
    x_tensor = torch.from_numpy(np.concatenate([winning_matchups, losing_matchups])).double()
    y_tensor = torch.from_numpy(np.concatenate([(data.WScore-data.LScore), (data.LScore-data.WScore)]).reshape((-1,1))).double()
    return TensorDataset(x_tensor, y_tensor)

In [9]:
fname = 'dataset.pt'
if os.path.isfile(fname):
    dataset=torch.load(fname, weights_only=False)
else:
    dataset = gen_dataset(data)
    torch.save(dataset, fname)

Generate the train/validation split

In [10]:
batch_size=200

generator = torch.Generator().manual_seed(20250217)
train_data, validation_data = torch.utils.data.random_split(dataset, [0.95, 0.05], generator=generator)
train_loader = DataLoader(train_data, batch_size=batch_size)
validation_loader = DataLoader(validation_data, batch_size=batch_size)

## The Model
Define the model. Combine the embeddings for the two teams, go to a hidden layer, and then output to a prediction if the first team won

In [11]:
class Model(nn.Module):
    def __init__(self, embedding_size, model_sizes, dropout, stats_size=14):
        super(Model, self).__init__()
        hid1, hid2 = model_sizes
        self.team_embedding = nn.Embedding(len(teams), embedding_size)
        self.program_embedding = nn.Embedding(len(programs), embedding_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(4*embedding_size+2*stats_size, hid1)
        self.fc2 = nn.Linear(hid1, hid2)
        self.score_fc = nn.Linear(hid2, 1)
        self.result_fc = nn.Linear(hid2, 1)
        self.double()

    def forward(self, x):
        program = self.program_embedding(x[:,0].int())
        team = self.team_embedding(x[:,1].int())
        opponent_program = self.program_embedding(x[:,2].int())
        opponent = self.team_embedding(x[:,3].int())
        matchup = self.dropout1(torch.cat([program, team, opponent_program, opponent, x[:,4:]], axis=1))
        hidden1 = self.dropout2(F.relu(self.fc1(matchup)))
        hidden2 = self.dropout3(F.relu(self.fc2(hidden1)))
        score = self.score_fc(hidden2)
        result = F.sigmoid(self.result_fc(hidden2))
        return score, result
        

In [12]:
model = Model(embedding_size=256, model_sizes=(64,64), dropout=0.25).to(device)

## Training the model

Define the training function

In [13]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

def train(data, model, loss_fn, optimizer, full_loss=True):
    size = len(data.dataset)
    model.train()
    for batch, (x, y) in enumerate(data):
        x = x.to(device)
        y = y.to(device)
        pred_score, pred_result = model(x)
        actual_result = (y > 0).double()
        score_loss = loss_fn(pred_score, y)
        result_loss = loss_fn(pred_result, actual_result)
        if full_loss:
            (score_loss + 10 * result_loss).backward()
        else:
            result_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            score_loss, result_loss, current = score_loss.item(), result_loss.item(), (batch + 1) * len(x)
            print(f"score loss: {score_loss:>7f}, result loss: {result_loss:>7f} [{current:>6d}/{size:>6d}]", end="\r")

Define the testing function

In [14]:
def test(data, model, loss_fn, label="Test"):
    size = len(data.dataset)
    num_batches = len(data)
    model.eval()
    score_loss, result_loss, correct = 0, 0, 0
    with torch.no_grad():
        for x, y in data:
            x = x.to(device)
            y = y.to(device)
            score_pred, result_pred = model(x)
            actual_result = (y > 0).double()
            score_loss += loss_fn(score_pred, y).item()
            result_loss += loss_fn(result_pred, actual_result).item()
            correct += ((result_pred >= 0.5) == (actual_result == 1)).type(torch.float).sum().item()
    score_loss /= num_batches
    result_loss /= num_batches
    correct /= size
    print(f"{label}: Accuracy: {(100*correct):>0.1f}%, Score loss: {score_loss:>8f}, Result loss: {result_loss:>8f}")

Train the model

In [15]:
n_epochs = 20
for i in range(n_epochs):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train: Accuracy: 69.6%, Score loss: 186.790477, Result loss: 0.200700
Validation: Accuracy: 69.1%, Score loss: 187.885319, Result loss: 0.202533
Epoch 1
Train: Accuracy: 71.6%, Score loss: 164.792488, Result loss: 0.189105
Validation: Accuracy: 71.0%, Score loss: 167.026238, Result loss: 0.191702
Epoch 2
Train: Accuracy: 73.1%, Score loss: 154.353310, Result loss: 0.181798
Validation: Accuracy: 72.0%, Score loss: 158.161302, Result loss: 0.185741
Epoch 3
Train: Accuracy: 74.1%, Score loss: 143.112581, Result loss: 0.174339
Validation: Accuracy: 72.9%, Score loss: 148.378959, Result loss: 0.179650
Epoch 4
Train: Accuracy: 74.8%, Score loss: 136.281057, Result loss: 0.168800
Validation: Accuracy: 73.6%, Score loss: 142.636722, Result loss: 0.175450
Epoch 5
Train: Accuracy: 75.4%, Score loss: 132.437193, Result loss: 0.165756
Validation: Accuracy: 74.0%, Score loss: 139.467806, Result loss: 0.172821
Epoch 6
Train: Accuracy: 75.7%, Score loss: 129.248784, Result loss: 0.164322
Vali

Fine tune with only the result

In [16]:
for i in range(5):
    print(f"Epoch {i}")
    train(train_loader, model, loss_fn, optimizer, full_loss=False)
    test(train_loader, model, loss_fn, label="Train")
    test(validation_loader, model, loss_fn, label="Validation")

Epoch 0
Train: Accuracy: 76.8%, Score loss: 115.481926, Result loss: 0.155981
Validation: Accuracy: 75.2%, Score loss: 125.829982, Result loss: 0.165315
Epoch 1
Train: Accuracy: 76.8%, Score loss: 115.541911, Result loss: 0.155909
Validation: Accuracy: 75.1%, Score loss: 125.846841, Result loss: 0.165318
Epoch 2
Train: Accuracy: 76.8%, Score loss: 115.723241, Result loss: 0.155893
Validation: Accuracy: 75.1%, Score loss: 125.939716, Result loss: 0.165373
Epoch 3
Train: Accuracy: 76.9%, Score loss: 116.276219, Result loss: 0.155669
Validation: Accuracy: 75.2%, Score loss: 126.375578, Result loss: 0.165395
Epoch 4
Train: Accuracy: 76.9%, Score loss: 117.308622, Result loss: 0.155401
Validation: Accuracy: 75.1%, Score loss: 127.072642, Result loss: 0.165350


With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [17]:
mens_tourney = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
womens_tourney = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
tourney = pd.concat([mens_tourney, womens_tourney])

tourney_dataset = gen_dataset(tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=batch_size)

In [18]:
test(tourney_loader, model, loss_fn, label="Tourney")

Tourney: Accuracy: 74.0%, Score loss: 133.884647, Result loss: 0.168533


When it comes to tournament results we get about 7 out of 10 results. The lower result is likely due to teams having increased pairity.

### Performance by year


In [19]:
for season in tourney.Season.unique():
    loader = DataLoader(gen_dataset(tourney[tourney.Season == season]), batch_size=batch_size)
    test(loader, model, loss_fn, label=f"{season} Tournament")

2003 Tournament: Accuracy: 65.6%, Score loss: 122.368316, Result loss: 0.192738
2004 Tournament: Accuracy: 67.2%, Score loss: 106.096459, Result loss: 0.182197
2005 Tournament: Accuracy: 76.6%, Score loss: 104.098361, Result loss: 0.170219
2006 Tournament: Accuracy: 68.8%, Score loss: 101.537153, Result loss: 0.202069
2007 Tournament: Accuracy: 76.6%, Score loss: 108.066266, Result loss: 0.158674
2008 Tournament: Accuracy: 81.2%, Score loss: 137.295102, Result loss: 0.153474
2009 Tournament: Accuracy: 71.1%, Score loss: 135.029373, Result loss: 0.168624
2010 Tournament: Accuracy: 76.4%, Score loss: 131.947142, Result loss: 0.168146
2011 Tournament: Accuracy: 72.7%, Score loss: 126.482828, Result loss: 0.159025
2012 Tournament: Accuracy: 78.1%, Score loss: 101.846042, Result loss: 0.136726
2013 Tournament: Accuracy: 71.5%, Score loss: 138.148228, Result loss: 0.169117
2014 Tournament: Accuracy: 73.1%, Score loss: 142.589412, Result loss: 0.147493
2015 Tournament: Accuracy: 81.9%, Score 

## Inspect the model
First what are the sizes of the smallest input and output weights

In [21]:
print(f"Program embedding min: {model.program_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"Team embedding min: {model.team_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"FC min: {model.result_fc.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")

Program embedding min: 2.693452
Team embedding min: 3.532826
FC min: 0.008307


Calculate the average gradient for each input feature

In [26]:
def feature_eval(model, data):
    model.eval()
    team_grads = torch.zeros(model.team_embedding.weight.shape[1]).to(device)
    program_grads = torch.zeros(model.program_embedding.weight.shape[1]).to(device)
    stats_grads = torch.zeros(28).to(device)
    size = len(data.dataset)
    for batch, (x, y) in enumerate(data):
        x = x.to(device)
        y = y.to(device)
        x.requires_grad = True
        _, pred_result = model(x)
        team_grads += torch.autograd.grad(model(x)[1].mean(), model.team_embedding.parameters())[0].sum(axis=0)
        program_grads += torch.autograd.grad(model(x)[1].mean(), model.program_embedding.parameters())[0].sum(axis=0)
        stats_grads += torch.autograd.grad(model(x)[1].mean(), x)[0].sum(axis=0)[4:]
    return program_grads/size, team_grads/size, stats_grads

In [27]:
program_weights, team_weights, stats_weights = feature_eval(model, train_loader)

In [30]:
program_weights, team_weights

(tensor([ 3.0420e-06,  1.1918e-06,  1.9409e-06,  3.5540e-07,  2.0039e-06,
          4.9824e-07,  4.8118e-07, -3.2863e-06, -1.6152e-06, -2.4931e-06,
          3.6131e-06, -1.4893e-06,  1.1040e-06,  9.6701e-08,  1.9751e-06,
          8.5273e-07, -1.1637e-07,  1.0694e-07, -9.8771e-07, -1.9795e-06,
         -5.0662e-07, -2.5285e-06, -2.3360e-06,  2.7965e-06, -1.6686e-06,
          1.0466e-06,  2.4983e-06, -1.9507e-08, -3.5374e-06,  2.9486e-06,
         -1.1413e-07,  2.7427e-06,  1.3036e-06,  8.0871e-07, -3.8985e-06,
          1.7463e-07, -2.0167e-06,  2.5618e-06, -1.0680e-06, -1.3626e-06,
         -9.1837e-07,  7.4027e-08, -8.9482e-07, -9.7630e-07, -1.9306e-06,
          3.3690e-06, -7.4923e-08,  5.7760e-07, -5.8939e-07,  6.0282e-07,
         -3.2900e-06,  9.0736e-07,  1.9954e-06, -1.0511e-06,  7.6808e-07,
          5.8643e-08,  2.1990e-07, -1.3258e-06, -2.4147e-06,  3.6410e-06,
         -2.4979e-06,  9.4662e-07, -9.4337e-07, -6.5976e-07,  9.1030e-07,
         -5.1883e-07, -4.0894e-07, -2.

In [29]:
for stat, weight in zip([f"W{stat}" for stat in teamStats] + [f"L{stat}" for stat in teamStats], stats_weights):
    print(f"{stat}:\t{weight:>4f}")

WScore:	1.883158
WFGM:	0.474314
WFGA:	0.099655
WFGM3:	0.884001
WFGA3:	-0.036584
WFTM:	0.803011
WFTA:	0.416839
WOR:	0.377015
WDR:	0.232381
WAst:	1.016505
WTO:	-0.312807
WStl:	0.980850
WBlk:	2.253306
WPF:	-0.283323
LScore:	-1.768581
LFGM:	-0.458999
LFGA:	0.039141
LFGM3:	-1.277252
LFGA3:	-0.011112
LFTM:	-0.481857
LFTA:	-0.353329
LOR:	-0.394182
LDR:	-0.068741
LAst:	-0.887679
LTO:	0.487246
LStl:	-0.343305
LBlk:	-2.790674
LPF:	-0.017583


## Generating the submission file
### Phase 1

Write the results

In [ ]:
with open('submission.csv', 'w') as f:
    f.write("ID,Pred\n")
    season=2025
    for league in ('M', 'W'):
        teams_to_test = sorted(teams[(teams.Season==season) & (teams.League==league)].TeamID.values)
        matchups = [(t1, t2) for t1 in teams_to_test for t2 in teams_to_test if t1 < t2]
        matchups_tensor = torch.Tensor(np.array(
            [np.concat([[teamMapping[(t1, season)], teamMapping[(t2, season)]],
                        teamStats.loc[(t1, season)].values,
                        teamStats.loc[(t2, season)].values])
             for (t1, t2) in matchups])).int().to(device)
        _, predictions = model(matchups_tensor)
        for (t1, t2), pred in zip(matchups, predictions):
            f.write(f"{season}_{t1.item()}_{t2.item()},{pred.item()}\n")

## Save the model

In [ ]:
torch.save(model.state_dict(), 'model.pth')